# Value Lost

A notebook to determine the value lost when applying a maintenance strategy defined in an asset model to the population.

Requested: greg.bell@essentialenergy.com.au
Author: gavin.treseder@essentialenergy.com.au

## Project Setup

In [1]:
import copy
import sys
import os
sys.path.append(os.path.dirname(os.getcwd()))

from IPython.display import clear_output
import numpy as np 
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
import scipy.stats as ss

from pof.system import System
from pof.interface.figures import make_sensitivity_fig, make_value_lost_fig, make_contour_plot
from pof.loader.asset_model_loader import AssetModelLoader
from pof.paths import Paths
from pof.units import scale_units_val

paths = Paths()

%matplotlib inline

# Load Data
## Load the Asset Model
The asset model needs to be loaded from the excel file and converted into a component object that execute simulations

In [2]:
asset_model_filename = "Asset Model.xlsx"
model_path = paths.demo_path + asset_model_filename

try:
    aml = AssetModelLoader()
    pof_data = aml.load(model_path)
    system = System.from_dict(pof_data['overhead_network'])
    comp = system.comp['pole']
    print("Asset Model loaded")
    pass
except:
    comp = Component.demo()
    print("Demo Model loaded")

Asset Model loaded


In [150]:
def rolling_mean(df, x_var:str, y_var:str, window=5):
    """ Returns a df with a rolling mean"""
    #TODO set this up so it work for
    #TODO move this to expected_cost

    df = (
        df.copy(deep=True)
        .rename(columns={'task':'source'})
        .dropna()
        .sort_values(by=['source', x_var, y_var])
    )

    # Reduce the window size if not enough values are passed
    window = max(1,min(len(df[x_var].unique()), window))

    # Generate a rolling average
    df = df.groupby(by=['source', y_var]).rolling(window).mean().drop([y_var], axis=1).reset_index().dropna()

    df['active'] = df['active'].astype(bool)

    return df


In [42]:
def calc_value_lost(df, x_var:str, y_var:str, z_var='cost_cumulative'):
    """ Takes a df of risk costs and returns a df with teh value lost
    # TODO move this to risk cost
    # TODO check y_var and x_var are the right way around
    """

    prefix = ["quantity", "cost"]
    suffix = ["_cumulative", '_lifecycle']
    cols = [f"{pre}{suf}" for pre in prefix for suf in suffix]

    # Calculate the total cost for each scenario
    df = df.groupby(by=[x_var, y_var])[cols].sum()

    # Calculate the value_lost at each interval
    df['value_lost'] = df[z_var] - df.groupby(by=[y_var])[z_var].transform('min') 

    return df.reset_index()

## Long run simulation 
Run a long simulation to repeat the process above for the key outputs without the interim steps

In [9]:
# Set the parameters in the model_units
model_units = 'months'
t_end = 1200
n_iterations = 1000
sens_vars = {
    'pole-consequence-cost': [1e4, 2e4, 3e4, 4e4, 5e4, 1e5, 2e5, 5e5, 1e6, 2e6, 5e6],
    'pole-task_group_name-groundline-t_interval': np.arange(12, 120, 6)
}

x_axis = 'pole-task_group_name-groundline-t_interval'
y_axis = 'pole-consequence-cost'
z_axis = 'value_lost'

window=5

#[1e4, 2e4, 3e4, 4e4, 5e4,7.5e4, 1e5, 2e5, 3e5, 4e5,5e5, 7.5e5, 1e6, 1.25e6, 1.5e6, 1.75e6, 2e6]

In [100]:
model_units = 'months'
t_end = 1200
n_iterations = 1000
sens_vars = {
    'pole-consequence-cost': [5e6],
    'pole-task_group_name-groundline-t_interval': np.arange(12, 18, 6)
}

x_axis = 'pole-task_group_name-groundline-t_interval'
y_axis = 'pole-consequence-cost'
z_axis = 'value_lost'

window=5
comp2.units = model_units
df_sens_chain2 = comp2.sensitivty_chain(sens_vars, t_end=t_end, n_iterations=n_iterations)

In [69]:
#df_te = calc_value_lost(df=df_rolling2, x_var=x_axis, y_var=y_axis, z_var='cost_lifecycle')
df_te['life'] = df_te['cost_cumulative'] / df_te['cost_lifecycle']
df_te

,pole-task_group_name-groundline-t_interval,pole-consequence-cost,quantity_cumulative,quantity_lifecycle,cost_cumulative,cost_lifecycle,value_lost,life
0,24.0,5000000.0,34.7520,0.037271,245899.60,264.328671,111.450724,930.279712
1,30.0,5000000.0,26.2212,0.028291,220453.47,237.971638,85.093691,926.385478
2,36.0,5000000.0,21.3046,0.023107,179156.88,194.294895,41.416948,922.087427
3,42.0,5000000.0,17.9388,0.019598,162974.79,178.071945,25.193997,915.218792
4,48.0,5000000.0,15.5800,0.017114,146824.52,161.318262,8.440314,910.154365
5,54.0,5000000.0,13.9708,0.015279,139727.54,152.877947,0.000000,913.981006
6,60.0,5000000.0,12.5948,0.013840,144662.04,159.077287,6.199340,909.382118
7,66.0,5000000.0,11.5444,0.012652,154615.04,169.439519,16.561572,912.508729
8,72.0,5000000.0,10.6516,0.011695,151560.02,166.474749,13.596801,910.408463
9,78.0,5000000.0,9.8628,0.010837,149531.96,164.420954,11.543006,909.445887


In [70]:
comp2.mc_timeline(t_end,n_iterations=n_iterations)
df_erc = comp2.expected_risk_cost_df()

In [169]:
prefix = ["quantity", "cost"]
suffix = ["", "_annual", "_cumulative", '_lifecycle']
cols = [f"{pre}{suf}" for pre in prefix for suf in suffix]

df_erc.groupby(by=["task", "active"])[cols].max()

quantity  quantity_annual  quantity_cumulative  \
task                  active                                                   
risk                  False        NaN              NaN                  NaN   
                      True       0.002         0.000002                0.062   
conditional_failure   False        NaN              NaN                  NaN   
                      True       0.027         0.000030                0.238   
functional_failure    False        NaN              NaN                  NaN   
                      True       0.002         0.000002                0.062   
inspection_groundline False        NaN              NaN                  NaN   
                      True       0.980         0.001081                6.507   
pole_saver_rod        False      0.000         0.000000                0.000   
                      True         NaN              NaN                  NaN   
termite_treatment     False        NaN              NaN                  NaN   
                      True       0.021         0.000023                0.122   

                              quantity_lifecycle     cost  cost_annual  \
task                  active                                             
risk                  False                  NaN      NaN          NaN   
                      True              0.000068  10000.0    11.028532   
conditional_failure   False                  NaN      NaN          NaN   
                      True              0.000262     94.5     0.104220   
functional_failure    False                  NaN      NaN          NaN   
                      True              0.000068     14.0     0.015440   
inspection_groundline False                  NaN      NaN          NaN   
                      True              0.007176     49.0     0.054040   
pole_saver_rod        False             0.000000      0.0     0.000000   
                      True                   NaN      NaN          NaN   
termite_treatment     False                  NaN      NaN          NaN   
                      True              0.000135      2.1     0.002316   

                              cost_cumulative  cost_lifecycle  
task                  active                                   
risk                  False               NaN             NaN  
                      True          310000.00      341.884489  
conditional_failure   False               NaN             NaN  
                      True             833.00        0.918677  
functional_failure    False               NaN             NaN  
                      True             434.00        0.478638  
inspection_groundline False               NaN             NaN  
                      True             325.35        0.358813  
pole_saver_rod        False              0.00        0.000000  
                      True                NaN             NaN  
termite_treatment     False               NaN             NaN  
                      True              12.20        0.013455

745.570707070707

In [77]:
comp2.plot_ms()

In [84]:
df_sens_chain2.loc[df_sens_chain2['task']=='risk']

,task,active,quantity,quantity_annual,quantity_cumulative,quantity_lifecycle,cost,cost_annual,cost_cumulative,cost_lifecycle,pole-task_group_name-groundline-t_interval,pole-consequence-cost
0,risk,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,12,5000000.0
1,risk,True,0.002,0.000002,0.050,0.000052,10000.0,10.484268,250000.0,262.106709,12,5000000.0
12,risk,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,18,5000000.0
13,risk,True,0.002,0.000002,0.044,0.000047,10000.0,10.661038,220000.0,234.542828,18,5000000.0
24,risk,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,24,5000000.0
25,risk,True,0.002,0.000002,0.050,0.000054,10000.0,10.701872,250000.0,267.546789,24,5000000.0
36,risk,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,30,5000000.0
37,risk,True,0.003,0.000003,0.034,0.000037,15000.0,16.329125,170000.0,185.063422,30,5000000.0
48,risk,False,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,36,5000000.0
49,risk,True,0.003,0.000003,0.035,0.000038,15000.0,16.141095,175000.0,188.312771,36,5000000.0


In [151]:
df_sens_chain2

x_var=x_axis
y_var=y_axis
window=3

df = (
    df_sens_chain2.copy(deep=True)
    .rename(columns={'task':'source'})
    .dropna()
    .sort_values(by=['source', x_var, y_var])
)

# Reduce the window size if not enough values are passed
window = max(1,min(len(df[x_var].unique()), window))

# # Generate a rolling average
# df = df.groupby(by=['source', y_var]).rolling(5).mean().drop([y_var], axis=1).reset_index().dropna()

# df['active'] = df['active'].astype(bool)

window

1

In [158]:
# Get ready for plotting
df_rolling2 = rolling_mean(df=df_sens_chain2, x_var=x_axis, y_var=y_axis, window=3)

df_total2 = calc_value_lost(df=df_rolling2, x_var=x_axis, y_var=y_axis, z_var='cost_lifecycle')
px.line(df_total2.reset_index(), x=x_axis, y='cost_lifecycle', color=y_axis)


,pole-task_group_name-groundline-t_interval,pole-consequence-cost,quantity_cumulative,quantity_lifecycle,cost_cumulative,cost_lifecycle,value_lost
0,24.0,10000.0,34.6664,0.037238,3435.51,3.693417,1.704491
1,24.0,20000.0,34.5406,0.037183,3912.18,4.215289,1.943904
2,24.0,30000.0,34.5774,0.037218,4542.95,4.895473,2.243473
3,24.0,40000.0,34.8330,0.037259,4860.73,5.202752,2.080916
4,24.0,50000.0,34.6028,0.037211,5382.61,5.803409,2.443126
...,...,...,...,...,...,...,...
149,102.0,200000.0,7.7614,0.008544,11062.42,12.173060,4.036331
150,102.0,500000.0,7.7672,0.008540,25360.23,27.828487,7.478799
151,102.0,1000000.0,7.7668,0.008539,50568.47,55.563101,16.193371
152,102.0,2000000.0,7.7308,0.008542,90337.38,99.816540,35.517158


In [10]:
# Run the simulation
comp.units = model_units

df_sens_chain = comp.sensitivty_chain(sens_vars, t_end=t_end, n_iterations=n_iterations)

In [160]:
# Get ready for plotting
df_rolling = rolling_mean(df=df_sens_chain, x_var=x_axis, y_var=y_axis, window=window)

df_total = calc_value_lost(df=df_rolling, x_var=x_axis, y_var=y_axis, z_var='cost_lifecycle')

In [12]:
make_contour_plot(df_total, x_axis=x_axis, y_axis=y_axis, z_axis=z_axis)

In [161]:
df_total

,pole-task_group_name-groundline-t_interval,pole-consequence-cost,quantity_cumulative,quantity_lifecycle,cost_cumulative,cost_lifecycle,value_lost
0,12.0,5000000.0,59.951,0.063631,279174.70,296.310641,184.772297
1,18.0,5000000.0,39.852,0.042654,308245.85,329.922059,218.383716
2,24.0,5000000.0,30.352,0.032384,252662.60,269.576072,158.037729
3,30.0,5000000.0,24.172,0.026090,217325.95,234.567101,123.028758
4,36.0,5000000.0,19.433,0.021599,172088.90,191.267482,79.729138
5,42.0,5000000.0,17.297,0.018729,151944.05,164.525478,52.987134
6,48.0,5000000.0,15.269,0.016736,101762.90,111.538344,0.000000
7,54.0,5000000.0,13.523,0.014839,171752.15,188.461319,76.922976
8,60.0,5000000.0,12.378,0.013667,136574.60,150.798686,39.260343
9,66.0,5000000.0,11.387,0.012426,136604.00,149.065910,37.527567


In [162]:
px.line(df_total.reset_index(), x=x_axis, y=z_axis, color=y_axis)

In [131]:
df_total.to_csv(paths.output_path + "interim_model_results_200804.csv")

# Match Asset Models to the population
## Load the population data

In [16]:
filename = "ACS - Poles - Consequence Model Output.csv"
print("File Loading...")
df_cons = pd.read_csv(paths.csv_path + filename)
clear_output()
df_cons.head()

,Unnamed: 0,ASSET_ID,Depot (ZSG Area),Pole_LU_NSWMajo,Pole_LU_NSWDeta,Pole_LU_ALUMMaj,Pole_LU_ALUMDet,Road Crossing Conductor Type,HV_LV Status,BushfirePriority,...,C_Environment_Rank,Site_Bushfire_Moderate_Probability,Site_Bushfire_Severe_Probability,C_Bushfire_Dollars,Site_Environment_Insignificant_Probability,Site_Environment_Minor_Probability,C_Environment_Dollars,C_Financial_Dollars,Total Consequence $,Radial(Y/N)
0,0,388,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2802.19134,21198.144867,NaN
1,1,389,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2802.57978,21198.533307,NaN
2,2,390,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2803.09692,21199.050447,NaN
3,3,391,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2803.76382,21199.717347,NaN
4,4,392,Cobar Depot,Grazing,Rangeland grazing,Grazing native vegetation,Grazing native vegetation,None,HV Only,P3,...,5,0.000715,0.000005,755.419255,0.0,0,0,2804.33010,21200.283627,NaN


## Prepare the population Data

In [17]:
# TODO turn ino a function so it works for mutliple y vars
pop_var = 'Total Consequence $'
sens_var = 'pole-consequence-cost'
uid_col = 'ASSET_ID'
agg_cols = ['Depot (ZSG Area)']

df_sens_var = pd.DataFrame(sens_vars[sens_var], columns= [sens_var])

# Fill with average val
df_cons[pop_var] = df_cons[pop_var].fillna(df_cons[pop_var].mean())

# Match to the closest value from the model
df = pd.merge_asof(
    df_cons.sort_values(pop_var),
    df_sens_var,
    left_on = pop_var,
    right_on = sens_var,
    direction = 'forward'
)

# Remove columsns that aren't required
col_to_keep = [uid_col] + agg_cols + [pop_var] + [sens_var]
df = df[col_to_keep]

# Count into sub populations
by = agg_cols + [sens_var]
df_pop = df.groupby(by=by)[uid_col].count().reset_index().rename(columns={uid_col:'asset_count'})
df_pop.head()

,Depot (ZSG Area),pole-consequence-cost,asset_count
0,Albury Depot,10000.0,2495
1,Albury Depot,20000.0,5011
2,Albury Depot,30000.0,4112
3,Albury Depot,40000.0,1083
4,Albury Depot,50000.0,79


## Combine the models together

In [18]:
#Combine the two files together
df_result = pd.merge(df_pop, df_total, on=sens_var)

# Calculte the value lost across the population
df_result['value_lost_pop'] = df_result['value_lost'] * df_result['asset_count']

# Caculate the value lost across ignoring sub populations
df_all = df_result.groupby(by=x_axis).sum()[['value_lost', 'value_lost_pop']].reset_index()
df_all[agg_cols] = "all"

# Calculate the value lost within sub populations
by = [x_axis] + agg_cols
df_agg = df_result.groupby(by=by).sum()[['value_lost', 'value_lost_pop']].reset_index()

# Combine them together
df_all = df_all.append(df_agg)

# Plot the results
make_value_lost_fig(df_all, x_axis=x_axis, y_axis='value_lost_pop', legend =agg_cols[0], title=f'Annual Value Lost')

In [32]:
comp.fm['fungal decay | internal'].consequence.__dict__

{'_name': 'consequence',
 '_units': 'years',
 'up_to_date': True,
 'cost': 5000000.0,
 'group': None}

## Steering Co Graphs

In [21]:
from pof.interface.figures import make_sensitivity_fig
df_plot = df_rolling.loc[df_rolling[y_axis] == 4e5]

# df_direct = df_plot.loc[~df_plot['source'].isin(['risk', 'total', 'direct'])].groupby(by=x_axis).sum()
# df_direct['source']='direct'
# df_plot = df_direct.append(df_plot)

# df_total = df_plot.loc[~df_plot['source'].isin(['total', 'direct'])].groupby(by=x_axis).sum()
# df_total['source']='total'
# df_plot = df_total.append(df_plot)

df_plot.rename(columns={x_axis : x_axis.split('-')[1]}, inplace=True)
make_sensitivity_fig(df_plot, var_name=x_axis, y_axis = 'cost_cumulative')

c:\Users\gtreseder\OneDrive - KPMG\Documents\3. Client\Essential Energy\Probability of Failure Model\pof\.venv\lib\site-packages\pandas\core\frame.py:4300: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



KeyError: 't_interval'

In [19]:
df_plot = df_total.copy(deep=True)
df_plot['pole-task_group_name-groundline-t_interval'] = df_plot['pole-task_group_name-groundline-t_interval'] / 12

px.line(df_plot.reset_index(), x=x_axis, y='cost_cumulative', color=y_axis, title = "Cumulative Cost for Consequences at different Inspection Intervals").show()
px.line(df_plot.reset_index(), x=x_axis, y='cost_lifecycle', color=y_axis, title = "Lifcycle Cost for Consequences at different Inspection Intervals").show()
px.line(df_plot.reset_index(), x=x_axis, y='value_lost', color=y_axis, title = "Value Lost for Consequences at different Inspection Intervals").show()

In [20]:
df_plot = df_total.copy(deep=True)
df_plot['pole-task_group_name-groundline-t_interval'] = df_plot['pole-task_group_name-groundline-t_interval'] / 12

px.line(df_plot.reset_index(), x=x_axis, y='cost_cumulative', color=y_axis, title = "Cumulative Cost for Consequences at different Inspection Intervals").show()
px.line(df_plot.reset_index(), x=x_axis, y='cost_lifecycle', color=y_axis, title = "Lifcycle Cost for Consequences at different Inspection Intervals").show()
px.line(df_plot.reset_index(), x=x_axis, y='value_lost', color=y_axis, title = "Value Lost for Consequences at different Inspection Intervals").show()

df_plots = df_all.copy(deep=True)
df_plots['pole-task_group_name-groundline-t_interval'] = df_plots['pole-task_group_name-groundline-t_interval'] / 12
make_value_lost_fig(df_plots, x_axis=x_axis, y_axis='value_lost_pop', legend =agg_cols[0], title=f'Annual Value Lost by Depot').show()

df_plot_all = df_all.loc[df_all['Depot (ZSG Area)'].isin(['Gunnedah Depot', 'Queanbeyan Depot', 'Junee Depot', 'Braidwood Depot'])]
df_plot_all['pole-task_group_name-groundline-t_interval'] = df_plot_all['pole-task_group_name-groundline-t_interval'] / 12
make_value_lost_fig(df_plot_all, x_axis=x_axis, y_axis='value_lost_pop', legend =agg_cols[0], title=f'Annual Value Lost by Depot').show()

<ipython-input-20-4c95b73a345d>:13: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



## 1 step version, which seems causes memory errors

In [ ]:
# df_sens_var = pd.DataFrame(sens_vars[y_var], columns= [y_var])

# # Fill with average val
# df_cons[pop_var] = df_cons[pop_var].fillna(df_cons[pop_var].mean())

# # Match to the closest value from the model
# df = pd.merge_asof(
#     df_cons.sort_values(pop_var),
#     df_sens_var,
#     left_on = pop_var,
#     right_on = y_var,
#     direction = 'forward'
# )

# df_plot = pd.merge(df, df_total.reset_index(), on=y_var).groupby(by=x_var).sum().reset_index()
# #y= ['list(set(df_plot) - set([sens_var, uid_col]))']
# y = ['quantity_annual', 'cost_annual', 'value_lost']

# px.line(df_plot, x=x_var, y=y)